# Evaluation TextGrad with TextualVerifier in Loss

In [ ]:
import re
import time
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

import textgrad as tg
from textgrad.engine import get_engine
from textgrad.variable import Variable
from textgrad.loss import TextLoss
from textgrad.optimizer import TextualGradientDescent
from textgrad.verifier import TextualVerifierWithTracker

## Load Dataset

In [ ]:
initial_solution = pd.read_csv("datasets/initial-solution.csv")
initial_solution

## Setup Engine

In [ ]:
engine = get_engine("gemini-1.5-pro")
tg.set_backward_engine("gemini-1.5-pro", override=True)

## Experiment Iterations

In [ ]:
def extract_answer(text):
    # Now extract correctly
    match = re.search(r"(?i)Answer\s*:\s*([A-D])", text, re.IGNORECASE)
    if match:
        return match.group(1).upper()
    return "Z" # Z -> means for None (Incorrect Answer)

In [ ]:
def extract_answer_llm(question, final_solution):
    prompt = f"""
    <Question>
    {question}
    </Question>
    
    <Solution>
    {final_solution}
    </Solution>

    <Task>
    Based on last step of solution, which letter (ABCD) selected in question options?
    Response MUST ONLY in 1 letter where LETTER is one of ABCD!
    </Task>
    """
    answer = engine.generate(prompt)
        
    return answer

In [ ]:
def evaluate(row_data, engine):
    try:
        match = initial_solution[initial_solution["id"] == row_data["id"]]
        if match.empty:
            return None  # or raise error
        formatted_question = match.iloc[0]["formatted_question"]
        
        solution = Variable(row_data["raw_solution"],
                            requires_grad=True,
                            role_description=f"Solution to the math question: {formatted_question}")
        loss_system_prompt = Variable("""You will evaluate a solution to a math question. 
                                        Do not attempt to solve it yourself, do not give a solution, 
                                        only identify errors. Be super concise.""",
                                        requires_grad=False,
                                        role_description="system prompt")

        optimizer = TextualGradientDescent([solution])
        
        loss = TextLoss(loss_system_prompt, engine=engine)
        loss_value = loss(solution) # Forward method in Loss Function
        original_loss = loss_value.value

        # Optimize
        loss_value.backward()
        optimizer.step()

        verification_task_prompts = [
            # Perspective 1: Rule-based verifier (objective, procedural)
            """
            Evaluate the calculation step strictly based on mathematical correctness and procedural rules. 
            If the step violates any algebraic or logical principle, replace it with the corrected version of that step only. 
            Do not proceed to solve the full problem.
            Please proceed output with 
            """,

            # Perspective 2: Teaching assistant (didactic, pedagogical)
            """
            Review the calculation step from the perspective of a teaching assistant helping a student learn. 
            If there's an error or suboptimal explanation, provide a corrected version that would best aid the student's understanding. 
            Focus only on the step in question, without solving the full problem.
            """,

            # Perspective 3: Exam grader (concise, evaluative)
            """
            Assess the calculation step as an exam grader would when evaluating a student's solution. 
            Identify whether the step is mathematically valid and aligns with standard problem-solving conventions. 
            If incorrect, rewrite only the flawed step to reflect what a student should have written to receive full credit. 
            Avoid expanding beyond the current step or solving the entire problem.
            """
        ]

        verifier = TextualVerifierWithTracker(
            verifier_engine=engine, 
            use_cot_generation=True,
            use_step_breakdown=True,
            verification_task_prompts=verification_task_prompts,
            enable_logging=False
        )

        instance = Variable(f"initial_solution: {row_data['raw_solution']}\nloss_value: {original_loss}",
                            requires_grad=False,
                            role_description="instance")

        optimizer_instruction = Variable("""You will optimize $initial_solution based on $loss_value. Be super concise.""",
                                        requires_grad=False,
                                        role_description="optimizer prompt")

        verified_result = verifier.verify(instance=instance,
                                        instruction=optimizer_instruction,
                                        calculation=solution)

        tracker_data = verifier.get_tracker() # Tracker data  

        final_solution_answer = extract_answer(verified_result.value)

        if final_solution_answer == "Z":
            final_solution_answer = extract_answer_llm(
                question=formatted_question, 
                final_solution=verified_result.value)  

        return {
            "id": row_data["id"],
            "source": row_data["source"],
            "subject": row_data["subject"],
            "question": formatted_question,
            "correct_answer": row_data["correct_answer"],
            "original_solution": row_data["raw_solution"],
            "original_loss": original_loss,
            "verified_loss": "",
            "optimized_solution": solution.value,
            "verified_optimized_solution": verified_result.value,
            "final_solution": verified_result.value,
            "final_solution_answer": final_solution_answer, 
            "success": True,
            "error_message": None,
            "processing_time_ms": tracker_data['processing_time_ms'],
            "total_llm_calls": tracker_data['total_llm_calls'],
            "total_input_tokens": tracker_data['total_input_tokens'],
            "total_output_tokens": tracker_data['total_output_tokens'],
            "result": tracker_data
        }
        
    except Exception as e:
        # Return error information
        return {
            "id": row_data["id"],
            "source": "",
            "subject": "",
            "question": "",
            "correct_answer": "",
            "original_solution": "",
            "original_loss": "",
            "verified_loss": "",
            "optimized_solution": "",
            "verified_optimized_solution": "",
            "final_solution": "",
            "final_solution_answer": "", 
            "success": False,
            "error_message": str(e),
            "processing_time_ms": 0,
            "total_llm_calls": 0,
            "total_input_tokens": 0,
            "total_output_tokens": 0,
            "result": {}
        }

In [ ]:
def run_evaluation():
    results = []
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=32) as executor:
        futures = [
            executor.submit(evaluate, row.to_dict(), engine) 
            for _, row in initial_solution[400:402].iterrows()  
        ]
        
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing"):
            try:
                result = future.result(timeout=None)
                if result is not None:
                    results.append(result)
            except Exception as e:
                print(f"Task failed: {e}")
                error_result = {
                    'success': False,
                    'error_message': str(e),
                    'result': {}
                }
                results.append(error_result)

    experiment_df = pd.DataFrame(results)
    print(f"Completed in {time.time() - start_time:.1f} seconds")
    experiment_df.to_csv('results/textgrad-tv-o.csv', index=False)
    
    return experiment_df

In [ ]:
run_evaluation()